In [1]:
from keras.datasets import fashion_mnist,mnist
import wandb
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import copy
import argparse
from types import SimpleNamespace


from utilities.NeuralNetwork import NN
from utilities.HelperFunctions import OneHotEncoder
from utilities.config import * # reading global variables 


def pre_process(x):
    '''
    reshape and normalized the data to bring to 0-1 scale.
    '''
    x=x.reshape(-1,784)
    x=x/255
    return x

def load_data(dataset=fmnist_dataset,split_size=valid_split_size):
    
    '''
    loads and returns data after doing train-valid split.
    '''
    if dataset==fmnist_dataset:
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    elif dataset==mnist_dataset:
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    
    x_train,x_valid=x_train[:int(len(x_train)*split_size)],x_train[int(len(x_train)*split_size):] #splitting train into train and valid
    y_train,y_valid=y_train[:int(len(y_train)*split_size)],y_train[int(len(y_train)*split_size):]

    x_train=pre_process(x_train)
    x_valid=pre_process(x_valid)
    x_test=pre_process(x_test) 
    
    one_hot=OneHotEncoder(10)
    y_train=one_hot.transform(y_train)
    y_valid=one_hot.transform(y_valid)
    y_test=one_hot.transform(y_test)
    
    
    return x_train,y_train,x_valid,y_valid,x_test,y_test

x_train,y_train,x_valid,y_valid,x_test,y_test=load_data(fmnist_dataset,1-valid_split_size)
    

In [2]:
sweep_config = {
    'method': 'bayes', #grid, random,#bayes
    'name' : 'Bayesian sweep cross entropy',
    'metric': {
      'name': 'valid accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [5,10]
        },
        'num_layers':{
            'values':[3,4,5]
        },
         'hidden_size':{
            'values':[32,64,128]
        },
        'weight_decay':{
            'values':[0,0.0005,0.5]
        },
         'learning_rate':{
            'values':[0.001,0.0001]
        },
         'optimizer': {
            'values': ['sgd','momentum','nag','rmsprop','adam','nadam']           
        },
        'batch_size': {
            'values': [16,32,64]
        },
         'weight_init':{
            'values':['normal','xavier']            
        },
        'activation': {
            'values': ['sigmoid','tanh','relu']
        }, 
        'loss': {
            'values': [entropy_loss]
        }, 
        
        
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='Assign_1_DL')

Create sweep with ID: no7tpeup
Sweep URL: https://wandb.ai/cs22m080/Assign_1_DL/sweeps/no7tpeup


In [3]:
def main():
    '''
    WandB calls main function each time with differnet combination.
    
    We can retrive the same and use the same values for our hypermeters.
    
    '''
    with wandb.init() as run:
        params={}
        params=dict(wandb.config)
        params=SimpleNamespace(**params)
        
        layers=[params.hidden_size]*params.num_layers
        run_name="opt_"+wandb.config.optimizer+"-ac_"+wandb.config.activation+"-batch_"+str(wandb.config.batch_size)\
                + "-nlayers_"+str(wandb.config.num_layers)+"-lr_"+str(wandb.config.learning_rate)
        wandb.run.name=run_name
        layers.append(10)
        obj=NN(784,layers,params)
        
        train_losses,tr_ac,valid_losses,val_ac=obj.train(x_train.T,y_train,x_valid.T,y_valid)
    
wandb.agent(sweep_id, function=main,count=100)
wandb.finish()


wandb: Agent Starting Run: 89twu0k2 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.0001
wandb: 	loss: cross_entropy
wandb: 	num_layers: 4
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.5
wandb: 	weight_init: normal
wandb: Currently logged in as: cs22m080. Use `wandb login --relogin` to force relogin


epoch 1 : train loss = 1.32 valid loss = 1.09 train accuracy = 68.05 valid accuracy = 68.35
epoch 2 : train loss = 1.08 valid loss = 1.07 train accuracy = 66.91 valid accuracy = 67.28
epoch 3 : train loss = 1.06 valid loss = 1.05 train accuracy = 68.25 valid accuracy = 68.93
epoch 4 : train loss = 1.05 valid loss = 1.04 train accuracy = 68.66 valid accuracy = 69.32
epoch 5 : train loss = 1.04 valid loss = 1.03 train accuracy = 68.77 valid accuracy = 69.25
epoch 6 : train loss = 1.03 valid loss = 1.02 train accuracy = 68.66 valid accuracy = 69.27
epoch 7 : train loss = 1.02 valid loss = 1.01 train accuracy = 68.51 valid accuracy = 69.50
epoch 8 : train loss = 1.01 valid loss = 1.01 train accuracy = 68.34 valid accuracy = 69.28
epoch 9 : train loss = 1.01 valid loss = 1.00 train accuracy = 68.17 valid accuracy = 69.28
epoch 10 : train loss = 1.01 valid loss = 1.00 train accuracy = 68.02 valid accuracy = 69.25


train accuracy,▅▁▆███▇▆▆▅
train loss,█▃▂▂▂▁▁▁▁▁
valid accuracy,▄▁▆▇▇▇█▇▇▇
valid loss,█▆▅▄▃▂▂▂▁▁
train accuracy,68.01667
train loss,1.0059
valid accuracy,69.25
valid loss,0.99902


wandb: Agent Starting Run: qjwujd1s with config:
wandb: 	activation: relu
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.0001
wandb: 	loss: cross_entropy
wandb: 	num_layers: 5
wandb: 	optimizer: nag
wandb: 	weight_decay: 0.5
wandb: 	weight_init: normal


/Users/sathish/Desktop/DL_assign/DL_Assignment_1/utilities/NeuralNetwork.py:61: RuntimeWarning: overflow encountered in matmul
  self.deltas[i]=np.matmul(self.weights[i+1].T,self.deltas[i+1])*self.activations.getDerivatives(inter_values[i])
/Users/sathish/Desktop/DL_assign/DL_Assignment_1/utilities/NeuralNetwork.py:61: RuntimeWarning: invalid value encountered in multiply
  self.deltas[i]=np.matmul(self.weights[i+1].T,self.deltas[i+1])*self.activations.getDerivatives(inter_values[i])


...

epoch 1 : train loss = 1.58 valid loss = 1.02 train accuracy = 61.95 valid accuracy = 62.57
epoch 2 : train loss = 0.84 valid loss = 0.71 train accuracy = 73.91 valid accuracy = 74.42
epoch 3 : train loss = 0.65 valid loss = 0.59 train accuracy = 78.52 valid accuracy = 78.50
epoch 4 : train loss = 0.55 valid loss = 0.53 train accuracy = 81.49 valid accuracy = 81.12
epoch 5 : train loss = 0.50 valid loss = 0.49 train accuracy = 83.42 valid accuracy = 82.48
epoch 6 : train loss = 0.46 valid loss = 0.46 train accuracy = 84.51 valid accuracy = 83.70
epoch 7 : train loss = 0.44 valid loss = 0.44 train accuracy = 85.29 valid accuracy = 84.43
epoch 8 : train loss = 0.42 valid loss = 0.42 train accuracy = 85.82 valid accuracy = 84.80
epoch 9 : train loss = 0.40 valid loss = 0.41 train accuracy = 86.26 valid accuracy = 85.17
epoch 10 : train loss = 0.39 valid loss = 0.40 train accuracy = 86.69 valid accuracy = 85.68


train accuracy,▁▄▆▇▇▇████
train loss,█▄▃▂▂▁▁▁▁▁
valid accuracy,▁▅▆▇▇▇████
valid loss,█▄▃▂▂▂▁▁▁▁
train accuracy,86.68519
train loss,0.38668
valid accuracy,85.68333
valid loss,0.39814


wandb: Agent Starting Run: 1ifgacr7 with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	loss: cross_entropy
wandb: 	num_layers: 5
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: xavier


epoch 1 : train loss = 0.93 valid loss = 0.54 train accuracy = 80.33 valid accuracy = 80.42
epoch 2 : train loss = 0.50 valid loss = 0.47 train accuracy = 83.07 valid accuracy = 82.33
epoch 3 : train loss = 0.45 valid loss = 0.44 train accuracy = 84.59 valid accuracy = 83.85
epoch 4 : train loss = 0.42 valid loss = 0.41 train accuracy = 85.54 valid accuracy = 84.78
epoch 5 : train loss = 0.40 valid loss = 0.40 train accuracy = 86.17 valid accuracy = 85.25


train accuracy,▁▄▆▇█
train loss,█▂▂▁▁
valid accuracy,▁▄▆▇█
valid loss,█▅▃▂▁
train accuracy,86.16852
train loss,0.40343
valid accuracy,85.25
valid loss,0.39885
